# Import Dependencies

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

# Extract our data files and store in data frames

In [31]:
csv_file1 = "qgdpstate0519_3.csv"
gdp_by_state_df = pd.read_csv(csv_file1)
gdp_by_state_df.head()

,"Table 1. Percent Change in Real Gross Domestic Product (GDP) by State and Region, 2017:Q1-2018:Q4",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,NaN,Seasonally adjusted at annual rates,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2017,NaN,NaN,NaN,2018,NaN,NaN,NaN,Rank 2018:Q4,NaN
2,NaN,Q1,Q2,Q3,Q4,Q1,Q2,Q3,Q4,NaN,NaN
3,United States,1.8,3.0,2.8,2.3,2.2,4.2,3.4,2.2,--,NaN
4,New England,0.6,2.8,3.1,-0.5,3.6,0.6,3.3,1.7,--,NaN


In [49]:
csv_file2 = "spi0619.csv"
personal_income_df = pd.read_csv(csv_file2)
personal_income_df.head()

,"Table 1. Personal Income, by State and Region, 2017:Q4-2019:Q1",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,"[Millions of dollars, seasonally adjusted at a...",NaN,NaN,NaN,NaN,NaN,[Seasonally adjusted at annual rate],NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2017,2018,NaN,NaN,NaN,2019,Percent change from preceding quarter1,NaN,NaN,NaN,NaN,Rank,NaN
2,NaN,Q4,Q1r,Q2r,Q3r,Q4r,Q1p,2018:Q1,2018:Q2,2018:Q3,2018:Q4,2019:Q1,2018:Q4 - 2019:Q1,NaN
3,United States,"17,090,705","17,313,151","17,460,221","17,647,510","17,825,728","17,975,691",5.3,3.4,4.4,4.1,3.4,--,NaN
4,New England,"963,895","979,868","981,773","992,031","998,048","1,004,541",6.8,0.8,4.2,2.4,2.6,--,NaN


# Clean data by selecting specific columns and dropping columns

In [53]:
new_gdp_by_state_df = gdp_by_state_df[['Table 1. Percent Change in Real Gross Domestic Product (GDP) by State and Region, 2017:Q1-2018:Q4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8']].copy()

new_gdp_by_state_df = new_gdp_by_state_df.rename(columns={"Table 1. Percent Change in Real Gross Domestic Product (GDP) by State and Region, 2017:Q1-2018:Q4": "state",
                                                          "Unnamed: 5": "FY18_Q1_GDP",
                                                          "Unnamed: 6": "FY18_Q2_GDP",
                                                          "Unnamed: 7": "FY18_Q3_GDP",
                                                          "Unnamed: 8": "FY18_Q4_GDP" })
new_gdp_by_state_df = new_gdp_by_state_df.drop([0, 1])
new_gdp_by_state_df.head()

,state,FY18_Q1_GDP,FY18_Q2_GDP,FY18_Q3_GDP,FY18_Q4_GDP
2,NaN,Q1,Q2,Q3,Q4
3,United States,2.2,4.2,3.4,2.2
4,New England,3.6,0.6,3.3,1.7
5,Connecticut,1.7,-3.6,9.0,1.8
6,Maine,2.0,2.5,3.1,0.7


In [63]:
new_personal_income_df = personal_income_df[['Table 1. Personal Income, by State and Region, 2017:Q4-2019:Q1',  'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']].copy()

new_personal_income_df = new_personal_income_df.rename(columns={"Table 1. Personal Income, by State and Region, 2017:Q4-2019:Q1": "state",
                                                          "Unnamed: 2": "FY18_Q1_income",
                                                          "Unnamed: 3": "FY18_Q2_income",
                                                          "Unnamed: 4": "FY18_Q3_income",
                                                          "Unnamed: 5": "FY18_Q4_income" })


new_personal_income_drop_rows_df = new_personal_income_df.drop([0, 1])

new_personal_income_drop_rows_df.head()

,state,FY18_Q1_income,FY18_Q2_income,FY18_Q3_income,FY18_Q4_income
2,NaN,Q1r,Q2r,Q3r,Q4r
3,United States,"17,313,151","17,460,221","17,647,510","17,825,728"
4,New England,"979,868","981,773","992,031","998,048"
5,Connecticut,"263,070","263,185","267,948","268,375"
6,Maine,"63,811","64,257","64,835","65,049"


# Connect to local database

In [60]:
rds_connection_string = "postgres:password@localhost:5432/gdp_saving_db"
engine = create_engine(f'postgresql://{rds_connection_string}')
engine.table_names()

[]

In [65]:
new_personal_income_drop_rows_df.to_sql(name='income', con=engine, if_exists='append', index=False)

In [62]:
new_gdp_by_state_df.to_sql(name='gdp', con=engine, if_exists='append', index=True)

# Check for tables

In [68]:
pd.read_sql_query('select * from income', con=engine).head()

,state,FY18_Q1_income,FY18_Q2_income,FY18_Q3_income,FY18_Q4_income
0,None,Q1r,Q2r,Q3r,Q4r
1,United States,"17,313,151","17,460,221","17,647,510","17,825,728"
2,New England,"979,868","981,773","992,031","998,048"
3,Connecticut,"263,070","263,185","267,948","268,375"
4,Maine,"63,811","64,257","64,835","65,049"


In [69]:
pd.read_sql_query('select * from gdp', con=engine).head()

,index,state,FY18_Q1_GDP,FY18_Q2_GDP,FY18_Q3_GDP,FY18_Q4_GDP
0,2,None,Q1,Q2,Q3,Q4
1,3,United States,2.2,4.2,3.4,2.2
2,4,New England,3.6,0.6,3.3,1.7
3,5,Connecticut,1.7,-3.6,9.0,1.8
4,6,Maine,2.0,2.5,3.1,0.7


# Confirm data in joined table

In [83]:
#joined_table_df = pd.read_sql_query('SELECT "gdp.state", "gdp.FY18_Q1_GDP", "gdp.FY18_Q2_GDP", "gdp.FY18_Q3_GDP", "gdp.FY18_Q4_GDP" FROM "gdp" FULL OUTER JOIN "income" ON "gdp.state" = "income.state" ORDER BY "gdp.state"', con=engine)
joined_table_df = pd.merge(new_personal_income_drop_rows_df, new_gdp_by_state_df, on='state', how='left')

joined_table_df.head()

,state,FY18_Q1_income,FY18_Q2_income,FY18_Q3_income,FY18_Q4_income,FY18_Q1_GDP,FY18_Q2_GDP,FY18_Q3_GDP,FY18_Q4_GDP
0,NaN,Q1r,Q2r,Q3r,Q4r,Q1,Q2,Q3,Q4
1,United States,"17,313,151","17,460,221","17,647,510","17,825,728",NaN,NaN,NaN,NaN
2,New England,"979,868","981,773","992,031","998,048",NaN,NaN,NaN,NaN
3,Connecticut,"263,070","263,185","267,948","268,375",NaN,NaN,NaN,NaN
4,Maine,"63,811","64,257","64,835","65,049",NaN,NaN,NaN,NaN
